In [1]:
import pandas as pd
import textwrap
import os
import json
import re
import math
import itertools

In [2]:
import pandas as pd
import re

df = pd.read_csv("results_by_category.csv")

df['metric_category'] = (df['metric_category']
                         .str.replace('_', ' ')          # underscores → spaces
                         .str.replace(' ', r'\\ '))      # insert line-break for wrapping
df['algorithm'] = df['algorithm'].str.replace(r'_(coarse|fine)$', '', regex=True)


In [4]:
body = []
for i, (cat, g_cat) in enumerate(df.groupby('metric_category', sort=False)):
    for j, (val, g_val) in enumerate(g_cat.groupby('category_value', sort=False)):
        for k, row in g_val.iterrows():
            ours = row['algorithm'].lower() == 'ta-hdbscan'
            cells = [
                rf'\multirow{{{len(g_cat)}}}{{*}}{{{cat}}}' if k == g_cat.index[0] and j == 0 else '',
                rf'\multirow{{{len(g_val)}}}{{*}}{{{val}}}' if k == g_val.index[0] else '',
                rf'\textbf{{{row["algorithm"]}}}' if ours else row['algorithm'],
                rf'\textbf{{{row["f1"]:.2f}}}'            if ours else f'{row["f1"]:.2f}',
                rf'\textbf{{{row["f1_as_pct_orac"]:.2f}}}' if ours else f'{row["f1_as_pct_orac"]:.2f}',
            ]
            body.append(' & '.join(cells) + r'\\')
        if val != g_cat['category_value'].iloc[-1]:
            body.append(r'\cline{2-5}')
    if i < df['metric_category'].nunique() - 1:
        body.append(r'\midrule')

# LaTeX table -------------------------------------------------------------
table = rf"""\begin{{table}}[t]
\vspace{{-1.1em}}
\centering
\caption{{Performance of HDBSCAN against baseline algorithms for different (true) stop types.}}
\label{{tab:performance_by_category}}
\small
\renewcommand{{\arraystretch}}{{0.95}}
\setlength{{\tabcolsep}}{{3pt}}
\setlength{{\extrarowheight}}{{0.25ex}}

\begin{{tabularx}}{{\linewidth}}{{%
  @{{}}>{{\raggedright\arraybackslash}}p{{2.1cm}}
  >{{\raggedright\arraybackslash}}p{{1.15cm}}
  >{{\raggedright\arraybackslash}}p{{1.55cm}}
  >{{\centering\arraybackslash}}p{{0.55cm}}
  >{{\centering\arraybackslash}}p{{1.05cm}}@{{}}}}
\toprule
\shortstack{{\textbf{{Stop}}\\\textbf{{category}}}} &
\textbf{{Value}} & \textbf{{Algorithm}} & \textbf{{F1}} &
\shortstack{{\textbf{{F1}}\\\textbf{{(\% of Oracle)}}}}\\
\midrule
{chr(10).join(body)}
\bottomrule
\end{{tabularx}}
\vspace{{-0.75em}}
\end{{table}}"""

print(table)

\begin{table}[t]
\vspace{-1.1em}
\centering
\caption{Performance of HDBSCAN against baseline algorithms for different (true) stop types.}
\label{tab:performance_by_category}
\small
\renewcommand{\arraystretch}{0.95}
\setlength{\tabcolsep}{3pt}
\setlength{\extrarowheight}{0.25ex}

\begin{tabularx}{\linewidth}{%
  @{}>{\raggedright\arraybackslash}p{2.1cm}
  >{\raggedright\arraybackslash}p{1.15cm}
  >{\raggedright\arraybackslash}p{1.55cm}
  >{\centering\arraybackslash}p{0.55cm}
  >{\centering\arraybackslash}p{1.05cm}@{}}
\toprule
\shortstack{\textbf{Stop}\\\textbf{category}} &
\textbf{Value} & \textbf{Algorithm} & \textbf{F1} &
\shortstack{\textbf{F1}\\\textbf{(\% of Oracle)}}\\
\midrule
\multirow{12}{*}{building\\ type} & \multirow{3}{*}{home} & lachesis & 0.50 & 72.97\\
 &  & \textbf{ta-hdbscan} & \textbf{0.55} & \textbf{81.28}\\
 &  & tadbscan & 0.53 & 80.02\\
\cline{2-5}
 & \multirow{3}{*}{park} & lachesis & 0.21 & 64.83\\
 &  & \textbf{ta-hdbscan} & \textbf{0.24} & \textbf{79.28}\\
 